In [ ]:
%pip install numpy scikit-learn

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer#bag of words y td idf
from sklearn.metrics.pairwise import cosine_similarity#similaridad de coseno de vectores, similaridad de  documentos (0 a 1)
from sklearn.naive_bayes import MultinomialNB, ComplementNB#modelos de clasificacion probabilistica
from sklearn.metrics import f1_score#combina precisión y recall(sensibilidad)

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado en sklearn
from sklearn.datasets import fetch_20newsgroups #devuelve el dataset de forma de diccionario claves: data(texto), target (numérica y nombre de las categorías )y file name(nombre del archivo)
import numpy as np

## Carga de datos

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))#solo aprende el contenido real
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [ ]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()#fila=un documento  columna=cada palabra  valor=peso de esa palabras en ese doc. Peso alto son raras y útiles para detectar

In [ ]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [ ]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador  (obtener el vocabulario y calcular el vector IDF) y transformar directamente los datos
# Parámetros por defecto relevantes:
#   - lowercase=True: convierte todo a minúsculas
#   - token_pattern='(?u)\\b\\w\\w+\\b': toma palabras de 2 o más caracteres alfanuméricos
#   - stop_words=None: no se remueven palabras vacías
#   - max_features=None: se incluyen TODAS las palabras (sin límite)
#   - max_df=1.0, min_df=1: no se filtran palabras por frecuencia   Podría ser min_df= ? filtra palabras que aparecen con poca frecuencia o max si lo hace con palabras muy comunes
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [ ]:
# recordar que las vectorizaciones por conteos son esparsas por ello sklearn convenientemente devuelve los vectores de documentos como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


X_train es una matriz dispersa (sparse matrix) en formato CSR (Compressed Sparse Row).No se almacenan los ceros, solo los valores no nulos

print(f'shape: {X_train.shape}')(n_documentos, n_palabras_del_vocabulario).

max_df=1.0, min_df=1 stop_words=None  max_features=None

In [ ]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [ ]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [ ]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [ ]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [ ]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [ ]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]# 0 porque es un solo documento una sola fila
#X_train[idx]Contiene los pesos TF-IDF de cada palabra en ese documento específico.

In [ ]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [ ]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 9019, 9016, 8748])

In [ ]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [ ]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [ ]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [ ]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)#Aplica suavizado (smoothing) para evitar log(0), Calcula probabilidades por clase,Calcula probabilidades de palabras dada la clase:

MultinomialNB()

Parámetros
alpha=1.0 Laplace smoothing
: evita que una palabra con frecuencia 0 "rompa" el cálculo de probabilidad
fit_prior=True  Aprende la probabilidad a priori de cada clase a partir de los datos.
class_prior=None Si no se da, usa las frecuencias de clases en
y_train
clases desbalanceadas, probar con ComplementNB


In [ ]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [ ]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación  es robusta al desbalance de clases.
#El promediado 'macro' es el promedio de los F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


# 1. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos. Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.

In [21]:
# IMPORTACIONES GENERALES
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd

**1 Preparación**

In [22]:
from sklearn.metrics.pairwise import cosine_similarity
# Cargar datos
newsgroups_train = fetch_20newsgroups(
    subset='train',
    remove=('headers', 'footers', 'quotes'),
    random_state=42
)

# Vectorizar con parámetros
tfidfvect = TfidfVectorizer(
    max_features=5000,      # Limito el  tamaño ya que decidí usar bigramas
    stop_words='english',
    lowercase=True,
    ngram_range=(1, 2),     # Unigramas y bigramas
    max_df=0.95,  #Ignoraré palabras muy comunes o sea alta frecuencia de aparición
    min_df=2  #ignoraré tokens que aparecen en menos de dos documentos
)

X_train = tfidfvect.fit_transform(newsgroups_train.data)

**2 Elegir los 5 documentos**

In [23]:
np.random.seed(42)
random_indices = np.random.choice(len(newsgroups_train.data), size=5, replace=False)# replace=false (los índices elegidos no se repiten)
print("Índices elegidos:", random_indices)

Índices elegidos: [7492 3546 5582 4793 3813]


**3 Definir la función de  similitud**

In [24]:
def analizar_similaridad(idx, X_train, data, target_names, target):
    print(f"\n" + "="*60)
    print(f"ANALIZANDO DOCUMENTO {idx} | Categoría: {target_names[target[idx]]}")
    print("="*60)
    print("TEXTO:")
    print(data[idx][:300] + "...\n")

    # Calcular similitud del coseno
    cossim = cosine_similarity(X_train[idx], X_train)[0]
    # Ordenar por similitud descendente y tomar los 5 más similares (excluyendo el original en posición 0) asumimos que el documento original (idx) es el más similar (posición 0)
    np.sort(cossim)[::-1]
    mostsim = np.argsort(cossim)[::-1][1:6]
    print("DOCUMENTOS MÁS SIMILARES:")
    for i in mostsim:
        print(f"  - Índice {i} | Sim: {cossim[i]:.3f} | Categoría: {target_names[target[i]]}")
        print(f"    Texto: {data[i][:150]}..........\n")

4 **Ejecución y visualización**

In [25]:
for idx in random_indices:
    analizar_similaridad(idx, X_train, newsgroups_train.data,
                        newsgroups_train.target_names, newsgroups_train.target)


ANALIZANDO DOCUMENTO 7492 | Categoría: comp.sys.mac.hardware
TEXTO:
Could someone please post any info on these systems.

Thanks.
BoB
-- 
---------------------------------------------------------------------- 
Robert Novitskey | "Pursuing women is similar to banging one's head
rrn@po.cwru.edu  |  against a wall...with less opportunity for reward" ...

DOCUMENTOS MÁS SIMILARES:
  - Índice 10935 | Sim: 0.514 | Categoría: comp.sys.mac.hardware
    Texto: Hey everybody:

   I want to buy a mac and I want to get a good price...who doesn't?  So,
could anyone out there who has found a really good deal on a..........

  - Índice 985 | Sim: 0.305 | Categoría: comp.sys.ibm.pc.hardware
    Texto: What is address interliving? and memmory modules  interliving?
Thanks in advance for the info.
Robert.

..........

  - Índice 7258 | Sim: 0.296 | Categoría: comp.sys.ibm.pc.hardware
    Texto: Hay all:

    Has anyone out there heard of any performance stats on the fabled p24t.
 I was wondering what it

In [26]:
from IPython.display import display

# Crear el DataFrame
df = pd.DataFrame({
    'DOCUMENTO': [7492, 3546, 5582, 4793, 3813],
    'CATEGORÍA': [
        'comp.sys.mac.hardware',
        'comp.os.ms-windows.misc',
        'misc.forsale',
        'talk.politics.guns',
        'rec.sport.hockey'
    ],
    '¿ALGÚN SIMILAR EN MISMA CATEGORÍA?': [
        '✅ Sí (el más similar)',
        '❌ No',
        '✅ Sí (2 de 5)',
        '✅ Sí (1 de 5, pero relevante)',
        '❌ No'
    ],
    '¿TIENE SENTIDO LA SIMILITUD?': [
        '✅ Sí — el más similar es coherente',
        '⚠️ Parcial — vocabulario técnico compartido',
        '✅ Sí — muy coherente con otros anuncios',
        '⚠️ Parcial — 1 muy bueno, otros débiles',
        '❌ No — similitudes parecen ruido léxico'
    ]
})

display(df)

,DOCUMENTO,CATEGORÍA,¿ALGÚN SIMILAR EN MISMA CATEGORÍA?,¿TIENE SENTIDO LA SIMILITUD?
0,7492,comp.sys.mac.hardware,✅ Sí (el más similar),✅ Sí — el más similar es coherente
1,3546,comp.os.ms-windows.misc,❌ No,⚠️ Parcial — vocabulario técnico compartido
2,5582,misc.forsale,✅ Sí (2 de 5),✅ Sí — muy coherente con otros anuncios
3,4793,talk.politics.guns,"✅ Sí (1 de 5, pero relevante)","⚠️ Parcial — 1 muy bueno, otros débiles"
4,3813,rec.sport.hockey,❌ No,❌ No — similitudes parecen ruido léxico


Para cada documento seleccionado, se identificaron los 5 documentos más similares según la similitud coseno. Luego, se verificó si alguno de ellos pertenecía a la misma categoría temática y si su contenido textual justificaba dicha similitud.(cuadro  inmediatamente anterior)

La similitud coseno basada en TF-IDF logra capturar relaciones temáticas en algunos casos, especialmente cuando el vocabulario es técnico y específico (como en anuncios de venta o consultas de hardware). Sin embargo, en textos con lenguaje más abstracto, coloquial o cultural (como el hockey o la política), la similitud se basa en coincidencias léxicas superficiales, lo que puede llevar a resultados poco intuitivos. Esto revela una limitación del modelo: no entiende el significado profundo, solo la co-ocurrencia de términos.


# 2. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámetros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

**1. Cargar datos (limpios)**





In [27]:
# Cargar datos sin headers, footers ni quotes
newsgroups_train = fetch_20newsgroups(
    subset='train',
    remove=('headers', 'footers', 'quotes'),
    random_state=42
)

newsgroups_test = fetch_20newsgroups(
    subset='test',
    remove=('headers', 'footers', 'quotes'),
    random_state=42
)

**2. Probar diferentes configuraciones del vectorizador**

In [28]:
#  Probar diferentes configuraciones
configs = [
    {  # Config 1: Unigramas, 5k features
        'max_features': 5000,
        'stop_words': 'english',
        'ngram_range': (1, 1),
        'max_df': 0.95,
        'min_df': 2,
        'sublinear_tf': False
    },
    {  # Config 2: Bigramas, 10k features
        'max_features': 10000,
        'stop_words': 'english',
        'ngram_range': (1, 2),
        'max_df': 0.95,
        'min_df': 2,
        'sublinear_tf': False
    },
    {  # Config 3: Bigramas + restrictiva
        'max_features': 5000,
        'stop_words': 'english',
        'ngram_range': (1, 2),
        'max_df': 0.90,
        'min_df': 3,
        'sublinear_tf': True
    },
    {  # Config 4: Bigramas + menos restrictiva en max_df
        'max_features': 10000,
        'stop_words': 'english',
        'ngram_range': (1, 2),
        'max_df': 0.98,   # dejo pasar más palabras útiles
        'min_df': 2,
        'sublinear_tf': False
    },
    {  # Config 5: Trigramas + menos restrictiva en max_df
        'max_features': 6000,
        'stop_words': 'english',
        'ngram_range': (1, 3),
        'max_df': 0.96,
        'min_df': 2,
        'sublinear_tf': False
    },
    {  #Configuac 6 en base a la 3 y luego de ver resultados parciales decido aplicar sublinear_tf:true  pero con mas features
      'max_features': 10000, # Más grande
      'stop_words': 'english',
      'ngram_range': (1, 2),
      'max_df': 0.95,
      'min_df': 2,
      'sublinear_tf': True   # ← cambio
}
]

**3. Entrenar y evaluar con MultinomialNB y ComplementNB**



In [30]:
results = []

for i, config in enumerate(configs):
    print(f"\n🔍 Probando configuración {i+1}: {config}")

    vectorizer = TfidfVectorizer(**config)
    X_train = vectorizer.fit_transform(newsgroups_train.data)
    X_test = vectorizer.transform(newsgroups_test.data)

    y_train = newsgroups_train.target
    y_test = newsgroups_test.target

    models = {
        'MultinomialNB': MultinomialNB(),
        'ComplementNB': ComplementNB()
    }

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        f1_macro = f1_score(y_test, y_pred, average='macro')
        results.append({
            'config': i+1,
            'vectorizer': config,
            'model': name,
            'f1_macro': f1_macro
        })
        print(f"  {name}: F1-score macro = {f1_macro:.3f}")


🔍 Probando configuración 1: {'max_features': 5000, 'stop_words': 'english', 'ngram_range': (1, 1), 'max_df': 0.95, 'min_df': 2, 'sublinear_tf': False}
  MultinomialNB: F1-score macro = 0.625
  ComplementNB: F1-score macro = 0.635

🔍 Probando configuración 2: {'max_features': 10000, 'stop_words': 'english', 'ngram_range': (1, 2), 'max_df': 0.95, 'min_df': 2, 'sublinear_tf': False}
  MultinomialNB: F1-score macro = 0.643
  ComplementNB: F1-score macro = 0.664

🔍 Probando configuración 3: {'max_features': 5000, 'stop_words': 'english', 'ngram_range': (1, 2), 'max_df': 0.9, 'min_df': 3, 'sublinear_tf': True}
  MultinomialNB: F1-score macro = 0.621
  ComplementNB: F1-score macro = 0.632

🔍 Probando configuración 4: {'max_features': 10000, 'stop_words': 'english', 'ngram_range': (1, 2), 'max_df': 0.98, 'min_df': 2, 'sublinear_tf': False}
  MultinomialNB: F1-score macro = 0.643
  ComplementNB: F1-score macro = 0.664

🔍 Probando configuración 5: {'max_features': 6000, 'stop_words': 'english',

**4. Mostrar los mejores resultados**

In [31]:
best = max(results, key=lambda x: x['f1_macro'])

print("\n" + "="*60)
print("🏆 MEJOR CONFIGURACIÓN ENCONTRADA")
print("="*60)
print(f"Configuración N°: {best['config']}")
print(f"Modelo: {best['model']}")
print(f"F1-score macro: {best['f1_macro']:.3f}")
print(f"Parámetros: {best['vectorizer']}")


🏆 MEJOR CONFIGURACIÓN ENCONTRADA
Configuración N°: 6
Modelo: ComplementNB
F1-score macro: 0.665
Parámetros: {'max_features': 10000, 'stop_words': 'english', 'ngram_range': (1, 2), 'max_df': 0.95, 'min_df': 2, 'sublinear_tf': True}


 CONCLUSIÓN  CLASIFICACIÓN CON NAÏVE BAYES

✅ ComplementNB superó consistentemente a MultinomialNB en todas las configuraciones, confirmando su robustez en problemas multiclase desbalanceados como 20newsgroups.

✅ La inclusión de bigramas (ngram_range=(1,2)) mejoró significativamente la representación textual, al capturar combinaciones de palabras con significado contextual (ej: “space mission”, “hard disk”, “gun control”) que los unigramas ignoran.

⚠️ Contrariamente a lo esperado, el uso de sublinear_tf=True —que aplica una transformación logarítmica a los términos— logró una leve mejora en el F1-score macro (0.665 vs 0.664), pero solo cuando se combinó con un vocabulario amplio (max_features=10000) y sin filtros agresivos (max_df=0.95, min_df=2).

 Esto sugiere que suavizar las frecuencias de términos ayuda a reducir el peso de palabras muy frecuentes sin eliminarlas, mejorando la discriminación entre categorías — especialmente útil en textos con alto ruido o variabilidad léxica.

# 3. Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente".

**1: Transposicion**

In [33]:
X_terms = X_train.T  # Forma: (n_terminos, n_documentos)

**2: Calcular similitud entre palabras**

In [34]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Similitud entre todas las palabras (matriz grande, pero manejable si limitamos vocabulario)
similarity_matrix = cosine_similarity(X_terms)  # (n_terminos, n_terminos)

** 3: Elegir 5 palabras manualmente (no al azar)**

'space', 'windows', 'hockey', 'god', 'graphics'

 **4: Función para encontrar palabras similares**

In [35]:
def palabras_similares(palabra, vectorizer, similarity_matrix, top_n=5):
    try:
        idx = vectorizer.vocabulary_[palabra]
    except KeyError:
        print(f"La palabra '{palabra}' no está en el vocabulario.")
        return

    # Obtener similitudes con todas las demás palabras
    similitudes = similarity_matrix[idx]

    # Obtener los índices de las palabras más similares (excluyendo ella misma)
    similares_idx = np.argsort(similitudes)[-top_n-1:-1][::-1]  # excluye a sí misma

    print(f"\nPalabra: '{palabra}'")
    print("Palabras más similares:")
    for i in similares_idx:
        palabra_sim = vectorizer.get_feature_names_out()[i]
        sim_val = similitudes[i]
        print(f"  - '{palabra_sim}' (sim: {sim_val:.3f})")

**5: Aplicar a 5 palabras elegidas manualmente**

In [38]:
# Palabras elegidas por su relevancia temática
palabras_clave = ['space', 'windows', 'hockey', 'god', 'graphics']

# Re-calculate X_train with the vectorizer used in the function
vectorizer = TfidfVectorizer(
    max_features=5000,
    stop_words='english',
    ngram_range=(1, 2),
    max_df=0.95,
    min_df=2,
    sublinear_tf=True
)

X_train_corrected = vectorizer.fit_transform(newsgroups_train.data)
X_terms = X_train_corrected.T

# Calcular matriz de similitud entre palabras
similarity_matrix = cosine_similarity(X_terms)

# Analizar cada una
for palabra in palabras_clave:
    palabras_similares(palabra, vectorizer, similarity_matrix, top_n=5)


Palabra: 'space'
Palabras más similares:
  - 'space station' (sim: 0.308)
  - 'sci space' (sim: 0.268)
  - 'nasa' (sim: 0.255)
  - 'space shuttle' (sim: 0.251)
  - 'shuttle' (sim: 0.217)

Palabra: 'windows'
Palabras más similares:
  - 'dos' (sim: 0.327)
  - 'ms windows' (sim: 0.277)
  - 'dos windows' (sim: 0.249)
  - 'ms' (sim: 0.238)
  - 'windows nt' (sim: 0.230)

Palabra: 'hockey'
Palabras más similares:
  - 'nhl' (sim: 0.266)
  - 'game' (sim: 0.215)
  - 'players' (sim: 0.200)
  - 'hockey league' (sim: 0.199)
  - 'ice' (sim: 0.195)

Palabra: 'god'
Palabras más similares:
  - 'jesus' (sim: 0.295)
  - 'christ' (sim: 0.262)
  - 'bible' (sim: 0.255)
  - 'faith' (sim: 0.252)
  - 'lord' (sim: 0.238)

Palabra: 'graphics'
Palabras más similares:
  - 'comp graphics' (sim: 0.327)
  - 'computer graphics' (sim: 0.244)
  - 'comp' (sim: 0.196)
  - '3d' (sim: 0.179)
  - 'graphic' (sim: 0.148)


In [41]:
from IPython.display import display

# Resultados
resultados_palabras = [
    {
        'Palabra': 'space',
        'Más similares': "'space station', 'sci space', 'nasa', 'space shuttle', 'shuttle'",
        'Observación': '✅ Coherente: términos técnicos y propios del dominio espacial'
    },
    {
        'Palabra': 'windows',
        'Más similares': "'dos', 'ms windows', 'dos windows', 'ms', 'windows nt'",
        'Observación': '✅ Coherente: términos de sistemas operativos y entornos Microsoft/DOS'
    },
    {
        'Palabra': 'hockey',
        'Más similares': "'nhl', 'game', 'players', 'hockey league', 'ice'",
        'Observación': '✅ Coherente: vocabulario deportivo, equipos y contexto de juego'
    },
    {
        'Palabra': 'god',
        'Más similares': "'jesus', 'christ', 'bible', 'faith', 'lord'",
        'Observación': '✅ Coherente: términos religiosos, cristianos, espirituales'
    },
    {
        'Palabra': 'graphics',
        'Más similares': "'comp graphics', 'computer graphics', 'comp', '3d', 'graphic'",
        'Observación': '✅ Coherente: dominio de gráficos por computadora y foros técnicos'
    }
]

df = pd.DataFrame(resultados_palabras)


display(df.style.set_properties(**{'text-align': 'left'}).set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'left')]}
]))

,Palabra,Más similares,Observación
0,space,"'space station', 'sci space', 'nasa', 'space shuttle', 'shuttle'",✅ Coherente: términos técnicos y propios del dominio espacial
1,windows,"'dos', 'ms windows', 'dos windows', 'ms', 'windows nt'",✅ Coherente: términos de sistemas operativos y entornos Microsoft/DOS
2,hockey,"'nhl', 'game', 'players', 'hockey league', 'ice'","✅ Coherente: vocabulario deportivo, equipos y contexto de juego"
3,god,"'jesus', 'christ', 'bible', 'faith', 'lord'","✅ Coherente: términos religiosos, cristianos, espirituales"
4,graphics,"'comp graphics', 'computer graphics', 'comp', '3d', 'graphic'",✅ Coherente: dominio de gráficos por computadora y foros técnicos


 CONCLUSIÓN DEL PUNTO 3 — SIMILITUD ENTRE PALABRAS

✅ La estrategia de transponer la matriz término-documento y calcular similitud entre palabras arrojó resultados más interpretables y semánticamente coherentes que la similitud entre documentos.
➡️ Mientras que en documentos la similitud a veces se basaba en coincidencias léxicas superficiales o ruido, aquí las palabras similares comparten claramente dominio temático

La elección del nivel de análisis (documento vs palabra) y la selección cuidadosa de parámetros o vocabulario pueden marcar una gran diferencia en la calidad e interpretabilidad de los resultados.